In [ ]:
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import numpy as np
import gpflow
gpflow.config.set_default_float(np.float32)
from shapely import geometry
import pointpats
import shapely
from time import time

from joblib import Parallel, delayed

from sgptools.utils.tsp import run_tsp # TSP/VRP solver for initial path planning
from sgptools.objectives import *
from sgptools.methods import get_method
from sgptools.kernels import get_kernel
from sgptools.kernels.attentive import *
from sgptools.utils.gpflow import get_model_params
from sgptools.utils.data import Dataset
from sklearn.metrics import pairwise_distances

np.random.seed(1234)
tf.random.set_seed(1234)

In [ ]:
# Method to extract lengthscales map from attentive non stationary kernel
def predict_lengthscales(X, lengthscales, kernel):
        preds = np.zeros(len(X))
        repre1 = kernel.get_representations(X)
        for i in range(len(lengthscales)):
                attention = tf.tensordot(repre1[:, i],
                                         tf.transpose(repre1[:, i]),
                                         axes=0)
                preds += np.diag(attention) * lengthscales[i]
        return preds

In [ ]:
# Load data
data = np.load("N17E073.npy")
plt.imshow(data.T, origin="lower")
plt.show()

dataset = Dataset(data=data, dtype=np.float32,
                  num_train=5000, num_test=20000)
X_train, y_train = dataset.get_train()
x_max, y_max = X_train.max(axis=0)
x_min, y_min = X_train.min(axis=0)
env = geometry.Polygon([[x_max, y_max], [x_min, y_max], [x_min, y_min], [x_max, y_min]])

lengthscales = np.linspace(1, 10, 10)

# Train GP/Kernel 
_, noise_variance, kernel, model = get_model_params(
    X_train=X_train, y_train=y_train, 
    kernel=get_kernel('Attentive')(lengthscales=lengthscales),
    optimizer='tf.Nadam',
    learning_rate=1e-2,
    max_steps=1000,
    return_model=True,
    verbose=False)

In [ ]:
# Plot GP prediction and lengthscale map
X_test, _ = dataset.get_test()
grid_x, grid_y = np.mgrid[min(X_test[:, 0]):max(X_test[:, 0]):100j, 
                          min(X_test[:, 1]):max(X_test[:, 1]):100j]
X_test = np.stack([grid_x, grid_y], axis=-1)
x_dim, y_dim = X_test.shape[:2]
X_test = X_test.reshape(-1, 2).astype(X_train.dtype)

mean, std = model.predict_f(X_test)
lengthscale_preds = predict_lengthscales(X_test, lengthscales, kernel)

# Create subplots
fig, axes = plt.subplots(1, 2, figsize=(10, 5), constrained_layout=True)

# First subplot — training data
sc1 = axes[0].imshow(mean.numpy().reshape(x_dim, y_dim).T, origin="lower")
axes[0].set_title("GP Predictions")
axes[0].set_aspect('equal')

# Second subplot — test data
sc2 = axes[1].imshow(lengthscale_preds.reshape(x_dim, y_dim).T, origin="lower")
axes[1].set_title("Lengthscale Predictions")
axes[1].set_aspect('equal')

# Shared colorbar
fig.colorbar(sc2, ax=axes, orientation='vertical', 
             fraction=0.05, pad=0.04, label='Lengthscale')

plt.show()


In [ ]:
# Get contour
grid_x, grid_y = X_test.reshape(x_dim, y_dim, 2)[:, :, 0], X_test.reshape(x_dim, y_dim, 2)[:, :, 1]
grid_z = lengthscale_preds.reshape(x_dim, y_dim)
cs = plt.contourf(grid_x, grid_y, grid_z, levels=1)
plt.gca().set_aspect('equal')
plt.colorbar()

In [ ]:
# Extract each polygon with holes from the contour map
polygons = []
levels = []
for contour_path, level in zip(cs.get_paths(), [2, 4, 8]): 
    for idx, cp in enumerate(contour_path.to_polygons()):
        x = cp[:,0]
        y = cp[:,1]
        polygon = geometry.Polygon([(i[0], i[1]) for i in zip(x,y)])

        if idx == 0:
            polygons.append(polygon)
            levels.append(level)
        else:
            if polygons[-1].intersects(polygon):
                # Remove holes
                polygons[-1] = polygons[-1].difference(polygon)
            else:
                # Save disjoint polygons
                polygons.append(polygon)
                levels.append(level)

sum_p = polygons[0]
for i in range(1, len(polygons)):
    sum_p = shapely.union(sum_p, polygons[i])

# Plot each polygon and sample points proportional to the lengthscale density of each polygon
pts_list = []
train_pts = []
num_train_pts = 10000
for l, p in zip(levels, polygons):
    x_ext, y_ext = p.exterior.xy
    plt.plot(x_ext, y_ext, c='C0')

    for interior in p.interiors:
        x_int, y_int = interior.xy
        plt.plot(x_int, y_int, color='C0')

    c_area = np.pi*((l/2)**2)
    size = int(np.ceil(p.area/c_area))*4
    pts = pointpats.random.poisson(p, size=size)
    t_pts = pointpats.random.poisson(p, size=size*100)

    if size == 1:
        pts_list.append([pts])
    else:
        pts_list.append(pts)
    train_pts.append(t_pts)

pts_list = np.concatenate(pts_list).astype(X_train.dtype)
train_pts = np.concatenate(train_pts).astype(X_train.dtype)
print(len(pts_list), len(train_pts))
plt.scatter(pts_list[:, 0], pts_list[:, 1], s=5, c='r')
plt.scatter(train_pts[:, 0], train_pts[:, 1], s=1, c='C0', alpha=0.1)
plt.gca().set_aspect('equal')
plt.show()

In [ ]:
method = get_method('ContinuousSGP')
sp_optimizer = method(
    len(pts_list), 
    train_pts, 
    kernel,
    noise_variance, 
    X_init=pts_list.astype(X_train.dtype),
)

# 4. Run the optimization
X_sol = sp_optimizer.optimize()
X_sol = X_sol.reshape(-1, 2)

# Plot each polygon and solution points
for l, p in zip(levels, polygons):
    x_ext, y_ext = p.exterior.xy
    plt.plot(x_ext, y_ext, c='C0')

    for interior in p.interiors:
        x_int, y_int = interior.xy
        plt.plot(x_int, y_int, color='C0')

plt.scatter(X_sol[:, 0], X_sol[:, 1], s=5, c='k')
plt.gca().set_aspect('equal')
plt.show()

In [ ]:
y_sol, _ = model.predict_f(X_sol)
_, _, _, model_sol = get_model_params(
    X_train=X_sol, y_train=y_sol, 
    kernel=kernel,
    max_steps=0,
    return_model=True,
    verbose=False)

mean, var = model_sol.predict_f(X_train)
sol_path, _ = run_tsp(X_sol, time_limit=15)
sol_lengthscales = predict_lengthscales(X_sol, lengthscales, kernel)


fig, axes = plt.subplots(1, 3, figsize=(10, 5), constrained_layout=True)

# First subplot — training data
sc1 = axes[0].scatter(X_train[:, 0], X_train[:, 1], c=mean.numpy())
axes[0].scatter(X_sol[:, 0], X_sol[:, 1], c='r', s=5)
axes[0].plot(sol_path[0][:, 0], sol_path[0][:, 1], c='r')
axes[0].set_title("Sol GP Predictions")
axes[0].set_aspect('equal')

# Second subplot — variance
sc2 = axes[1].scatter(X_train[:, 0], X_train[:, 1], c=var.numpy())
axes[1].scatter(X_sol[:, 0], X_sol[:, 1], c='r', s=5)
axes[1].set_title("Sol GP Variance")
axes[1].set_aspect('equal')

# Fix colorbar size
cbar = fig.colorbar(sc2, ax=axes[1], fraction=0.046, pad=0.04)

# Third subplot — lengthscale
axes[2].scatter(X_test[:, 0], X_test[:, 1], c=lengthscale_preds)
axes[2].scatter(X_sol[:, 0], X_sol[:, 1], c='r', s=5)
axes[2].set_title("Lengthscale Map")
axes[2].set_aspect('equal')

for pt, d in zip(X_sol, sol_lengthscales):
    circle = patches.Circle(pt, 
                            d/2, 
                            edgecolor='k', 
                            facecolor='w', 
                            alpha=0.3)
    axes[2].add_patch(circle)

axes[2].set_xlim(axes[0].get_xlim())
axes[2].set_ylim(axes[0].get_ylim())

plt.show()

In [ ]:
locs = np.array([loc.centroid.coords[0] for loc in sol])
X_sol = locs.astype(X_train.dtype)
y_sol, _ = model.predict_f(X_sol)
_, _, _, model_sol = get_model_params(
    X_train=X_sol, y_train=y_sol, 
    kernel=kernel,
    max_steps=0,
    return_model=True,
    verbose=False)

mean, var = model_sol.predict_f(X_train)
sol_path = [locs]
sol_lengthscales = predict_lengthscales(X_sol, lengthscales, kernel)


fig, axes = plt.subplots(1, 3, figsize=(10, 5), constrained_layout=True)

# First subplot — training data
sc1 = axes[0].scatter(X_train[:, 0], X_train[:, 1], c=mean.numpy())
axes[0].scatter(X_sol[:, 0], X_sol[:, 1], c='r', s=5)
axes[0].plot(sol_path[0][:, 0], sol_path[0][:, 1], c='r')
axes[0].set_title("Sol GP Predictions")
axes[0].set_aspect('equal')

# Second subplot — variance
sc2 = axes[1].scatter(X_train[:, 0], X_train[:, 1], c=var.numpy())
axes[1].scatter(X_sol[:, 0], X_sol[:, 1], c='r', s=5)
axes[1].set_title("Sol GP Variance")
axes[1].set_aspect('equal')

# Fix colorbar size
cbar = fig.colorbar(sc2, ax=axes[1], fraction=0.046, pad=0.04)

# Third subplot — lengthscale
axes[2].scatter(X_test[:, 0], X_test[:, 1], c=lengthscale_preds)
axes[2].scatter(X_sol[:, 0], X_sol[:, 1], c='r', s=5)
axes[2].set_title("Lengthscale Map")
axes[2].set_aspect('equal')

for pt, d in zip(X_sol, sol_lengthscales):
    circle = patches.Circle(pt, 
                            d/2, 
                            edgecolor='k', 
                            facecolor='w', 
                            alpha=0.2)
    axes[2].add_patch(circle)

axes[2].set_xlim(axes[0].get_xlim())
axes[2].set_ylim(axes[0].get_ylim())

plt.show()

In [ ]:
import numpy as np
from joblib import Parallel, delayed
from shapely.ops import unary_union as union_all


def coverage_area(polygons, env):
    """
    Get effective coverage area from selected sensing locations.

    Parameters
    ----------
    polygons : Sequence[shapely.geometry.base.BaseGeometry]
        Polygons representing the observable sensing area at each sensing location.
    env : shapely.geometry.Polygon
        Polygon representing the extent of the environment.

    Returns
    -------
    float
        The area of the union of `polygons` clipped to `env`.
    """
    if not polygons:
        return 0.0

    union_poly = union_all(polygons)
    inside = env.intersection(union_poly)
    return inside.area


class PriorityQueue:
    """Max-heap implemented via negative priorities on a min-heap."""
    def __init__(self, indices, gains):
        gains = np.asarray(gains, dtype=float)
        self.pq = [(-gains[i], int(idx)) for i, idx in enumerate(indices)]
        import heapq
        heapq.heapify(self.pq)
        self._heapq = heapq

    def pop(self):
        return self._heapq.heappop(self.pq)

    def add(self, idx, gain):
        self._heapq.heappush(self.pq, (-float(gain), int(idx)))

    def __len__(self):
        return len(self.pq)


def lazy_greedy_until_coverage(polygons, env, target_fraction,
                               n_jobs=-1, max_selections=None):
    """
    Lazy greedy maximization of coverage area until a fraction of the
    environment is covered.

    Parameters
    ----------
    polygons : Sequence[shapely.geometry.Polygon]
        One polygon per sensing location.
    env : shapely.geometry.Polygon
        Environment polygon (coverage is clipped to this).
    target_fraction : float
        Fraction of the environment area to cover in [0, 1].
        The algorithm stops once coverage >= target_fraction * env.area.
    n_jobs : int, optional
        Number of CPU cores for parallel initialization (-1 = all cores).
    max_selections : int or None, optional
        Hard cap on the number of selected locations.
        If None, defaults to len(polygons).

    Returns
    -------
    selected : list[int]
        Indices of selected sensing locations in `polygons`.
    gains : list[float]
        Marginal coverage gain at each selection step.
    final_area : float
        Total covered area after stopping.
    final_union : shapely.geometry.base.BaseGeometry or None
        Union geometry of selected polygons clipped to `env`.
        None if nothing was selected.
    """
    n = len(polygons)
    idxs = np.arange(n, dtype=int)

    if max_selections is None:
        max_selections = n

    env_area = env.area
    if env_area <= 0:
        # Degenerate environment; nothing to cover
        return [], [], 0.0, None

    target_area = target_fraction * env_area

    # --------------- Step 1: parallel initialization (A = ∅) ----------
    # For A = ∅, gain for y is coverage_area({y}, env)
    if n_jobs is None or n_jobs == 1:
        init_gains = np.array(
            [coverage_area([polygons[i]], env) for i in idxs],
            dtype=float,
        )
    else:
        init_gains = np.array(
            Parallel(n_jobs=n_jobs)(
                delayed(coverage_area)([polygons[i]], env)
                for i in idxs
            ),
            dtype=float,
        )

    pq = PriorityQueue(idxs, init_gains)

    # Selected set bookkeeping
    selected_mask = np.zeros(n, dtype=bool)
    selected = []
    gains = []

    # Current union and area of selected set A
    current_union = None
    current_area = 0.0

    # ----------------- Step 2: lazy greedy loop -----------------------
    while (current_area < target_area
           and len(selected) < max_selections
           and len(pq) > 0):

        best_gain = float("-inf")
        best_idx = None

        while True:
            if len(pq) == 0:
                break

            neg_prev_gain, idx = pq.pop()

            # Skip if already selected
            if selected_mask[idx]:
                continue

            # If we pop the same index twice within this outer loop,
            # then its up-to-date gain is ≥ all others; accept it.
            if best_idx == idx:
                break

            # ---- Recompute true marginal gain given current A ----
            poly_y = polygons[idx]
            if current_union is None:
                new_union = poly_y
            else:
                new_union = current_union.union(poly_y)

            clipped = new_union.intersection(env)
            new_area = clipped.area
            gain = new_area - current_area

            # Push updated gain back into PQ
            pq.add(idx, gain)

            # Track best candidate this outer iteration
            if gain > best_gain:
                best_gain = gain
                best_idx = idx
            elif gain == best_gain and best_gain == 0.0:
                # If everything is effectively zero, accept first
                best_gain = gain
                best_idx = idx
                break

        if best_idx is None:
            # No viable candidates left
            break

        # ----------------- Commit best selection --------------------
        idx = best_idx
        poly_y = polygons[idx]

        if current_union is None:
            current_union = poly_y
        else:
            current_union = current_union.union(poly_y)

        clipped = current_union.intersection(env)
        current_union = clipped          # store clipped union
        current_area = clipped.area

        selected_mask[idx] = True
        selected.append(idx)
        gains.append(best_gain)

        # Optional: early numerical-stop if additional gain is negligible
        if current_area >= target_area:
            break

    return selected, gains, current_area, current_union


selected_idxs, gains, total_area, union_geom = lazy_greedy_until_coverage(
    polygons=candidates,
    env=env,
    target_fraction=0.99,
    n_jobs=-1,  # use all cores for initialization
)

In [ ]:
locs = np.array([loc.centroid.coords[0] for loc in np.array(candidates)[selected_idxs]])
X_sol = locs.astype(X_train.dtype)
y_sol, _ = model.predict_f(X_sol)
_, _, _, model_sol = get_model_params(
    X_train=X_sol, y_train=y_sol, 
    kernel=kernel,
    max_steps=0,
    return_model=True,
    verbose=False)

mean, var = model_sol.predict_f(X_train)
sol_path, _ = run_tsp(X_sol, time_limit=20)
sol_lengthscales = predict_lengthscales(X_sol, lengthscales, kernel)

fig, axes = plt.subplots(1, 3, figsize=(10, 5), constrained_layout=True)

# First subplot — training data
sc1 = axes[0].scatter(X_train[:, 0], X_train[:, 1], c=mean.numpy())
axes[0].scatter(X_sol[:, 0], X_sol[:, 1], c='r', s=5)
axes[0].plot(sol_path[0][:, 0], sol_path[0][:, 1], c='r')
axes[0].set_title("Sol GP Predictions")
axes[0].set_aspect('equal')

# Second subplot — variance
sc2 = axes[1].scatter(X_train[:, 0], X_train[:, 1], c=var.numpy())
axes[1].scatter(X_sol[:, 0], X_sol[:, 1], c='r', s=5)
axes[1].set_title("Sol GP Variance")
axes[1].set_aspect('equal')

# Fix colorbar size
cbar = fig.colorbar(sc2, ax=axes[1], fraction=0.046, pad=0.04)

# Third subplot — lengthscale
axes[2].scatter(X_test[:, 0], X_test[:, 1], c=lengthscale_preds)
axes[2].scatter(X_sol[:, 0], X_sol[:, 1], c='r', s=5)
axes[2].set_title("Lengthscale Map")
axes[2].set_aspect('equal')

for pt, d in zip(X_sol, sol_lengthscales):
    circle = patches.Circle(pt, 
                            d/2, 
                            edgecolor='k', 
                            facecolor='w', 
                            alpha=0.2)
    axes[2].add_patch(circle)

axes[2].set_xlim(axes[0].get_xlim())
axes[2].set_ylim(axes[0].get_ylim())

plt.show()